# Study of Probleam and Model decision  
Estimation of loan approval based on persons personal attributes  
Models Probleam is calssification  
Performance Measures are AUC & ROC(False Negatives, False Positives, Precision, Recall, threshold, Confusion Matrix), Balance Accurancy  
Useful Models include logistic Regression, Random Forest, SVM and KNN  

In [2]:
# Importing required framework libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Import required DataPipeLine libraries
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Import processing and evaluation libraries
import scipy.stats as stats #for qq plots
from sklearn.model_selection import train_test_split, learning_curve, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.metrics import plot_confusion_matrix, plot_precision_recall_curve, confusion_matrix, classification_report

# Import Model libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
#from lightgbm import LGBMClassifier #Light Gradient Boosting Machine based on Decision Tree

# Study of Data  
the target is decided as loan_status column   
It can be seen that person_home_ownership and cb_person_default_on_file are catogorical-Ordinal variable  
It can be seen that loan_intent and loan_grade are catogorical-nominal variables  
There 165 duplicate entries in the DataSet  
person_emp_length(3%) and loan_int_rate(11%) have null value in them  
Majority of the features are positive(left) skewed and also seem to have outliers  
They were outliers in person_age, person_income and person_emp_length, outlier should be based on these columns 

In [3]:
# Read the Data
df = pd.read_csv('credit_risk_dataset.csv')
print(">>Shape of Original Data: ",df.shape)
df.head(5)

>>Shape of Original Data:  (32581, 12)


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


# Train Test split  
Drop any duplicates in the dataset to avoid overfitting  
use stratify and shuffel for preserving and proper distribution of data  

In [4]:
# Train Test split of Data for generalised error with stratify and shuffel to maintain the distribution quality
X, X_test, y, y_test = train_test_split(df.drop('loan_status', axis=1), df['loan_status'],
                                        random_state=0,  test_size=0.2, stratify=df['loan_status'],
                                        shuffle=True)

In [5]:
# Check Splits for Distributions in the Train and Test Data with Respect to Original data
print("Normalized df data: \n",df['loan_status'].value_counts(normalize=True))
print("Normalized Train Split: \n",y.value_counts(normalize=True))
print("Normalized Train Split: \n",y_test.value_counts(normalize=True))

Normalized df data: 
 0    0.781836
1    0.218164
Name: loan_status, dtype: float64
Normalized Train Split: 
 0    0.781845
1    0.218155
Name: loan_status, dtype: float64
Normalized Train Split: 
 0    0.781801
1    0.218199
Name: loan_status, dtype: float64


# Data Prerpocessing  
Fill any missing values in the catogorical varibales. #Not Done  
Do encoding if needed for catogorical variables. #Not Done  
Data Preprocessing for eliminating obiviously false data  
Missing Data handling using best guess and KNN(or alternativly Iterative imputer). #Not Done      
Outlier handeling using IQR     
Feature Selection using Variance threshold and Correlation matrix. #Not Done  
Convertion of Catogorical to numerical using One-Hot and Ordinal encodings  
Develop the pre-processing model to fit and transform the test data also based on estimates from the training data  
No Additional steps for Random Forest  
Future transformation using log and scaling using Standarzation for logistic and SVM   
Scale the data using Min_Max for KNN 

## Study of Training Data    
It can be seen that person_home_ownership and cb_person_default_on_file are catogorical-Ordinal variable  
It can be seen that loan_intent and loan_grade are catogorical-nominal variables  
There 165 duplicate entries in the DataSet  
person_emp_length(3%) and loan_int_rate(11%) have null value in them  
Majority of the features are positive(left) skewed and also seem to have outliers  
They were outliers in person_age, person_income and person_emp_length, outlier should be based on these columns 

In [6]:
# Read the Data
print(">>Shape of Training Data: ",X.shape)
X.head(5)

>>Shape of Training Data:  (26064, 11)


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
20079,35,38000,MORTGAGE,2.0,MEDICAL,B,15000,10.99,0.39,N,5
22038,29,75000,RENT,2.0,HOMEIMPROVEMENT,B,7000,11.71,0.09,N,9
14429,26,106000,MORTGAGE,9.0,EDUCATION,D,10000,15.33,0.09,Y,3
4362,22,21000,RENT,5.0,MEDICAL,B,4800,10.75,0.23,N,3
28058,31,168000,MORTGAGE,2.0,EDUCATION,B,15500,9.25,0.09,N,9
